# Discovering Sentinel-2 products

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import visualize_S2_products, visualize_S2_products_v2

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

import geopandas as gpd
import pandas as pd
pd.set_option('display.max_colwidth', None)
import folium
import folium.plugins
import logging
import numpy as np

import shapely
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
# file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
shp = gpd.read_file(file_aoi).geometry[0]

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    platformname="Sentinel-2",
    date=("20230901", "20230920"),
    producttype="S2MSI1C",
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')

In [ ]:
def has_overlap(geom1, geom2, tolerance=0.01):
    intersection = geom1.intersection(geom2)
    return intersection.area / min(geom1.area, geom2.area) >= (1 - tolerance)


# Group the products by geometry with overlap tolerance
groups = []
for i, row in gdf.iterrows():
    geom1 = row['geometry']
    # group = [row['title']]
    group = [i]
    for i, inner_row in gdf.iterrows():
        geom2 = inner_row['geometry']
        if geom1 != geom2 and has_overlap(geom1, geom2, tolerance=0.02):
            # group.append(inner_row['title'])
            group.append(i)
    if len(group)>0:
        groups.append(group)

# Remove duplicate groups and sort
unique_groups = []
for group in groups:
    group = list(set(group))
    group.sort()
    if group not in unique_groups:
        unique_groups.append(group)

# Print the grouped products
# for i, group in enumerate(unique_groups):
for i,it in enumerate(unique_groups):
    print(f"Group {i+1}: {it}")

In [ ]:
# gdf.geometry[1:3].explore()
# gdf[gdf.s2datatakeid=='GS2A_20230908T110621_042891_N05.09'].geometry.explore()
shapely.geometry.mapping(gdf[gdf.s2datatakeid=='GS2A_20230908T110621_042891_N05.09'].unary_union)
# np.unique(gdf.s2datatakeid)
for g in gdf.groupby('s2datatakeid'):
    print(len(g[1]))

In [ ]:
print(gdf.iloc[0])

In [ ]:
print(gdf.loc[0:15, ['beginposition', 's2datatakeid', 'tileid']])

In [ ]:
len(np.unique(gdf.geometry.astype('str')))

In [ ]:
toto = gdf.groupby(['s2datatakeid'])
idx = toto.groups['GS2A_20230901T112121_042791_N05.09']

toto.get_group(idx)

## Visualize by groups

In [ ]:
visualize_S2_products(gdf, gj_aoi)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 5]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')